<a href="https://colab.research.google.com/github/outtaflames/HW3/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
__author__ = 'Chris Potts'


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        The temperature parameter. Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        spk = self.speaker().T
        return rownorm(spk * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Examples"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses']
    states = ['r1', 'r2']
    lex = pd.DataFrame([
        [0.0, 1.0],
        [1.0, 1.0]], index=msgs, columns=states)

In [4]:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0]], index=msgs, columns=states)
    

In [7]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.33, 0.33, 0.33], costs=[0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.00,1,0,0
glasses,0.00,0,1,0
mustache,0.00,1,0,0
prior,0.33,0.33,0.33,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
mustache,0.0,1.0,0.0



Pragmatic speaker


,hat,glasses,mustache
r1,1.000000,0.0,0.000000
r2,0.333333,0.0,0.666667
r3,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3
hat,0.75,0.25,0.0
glasses,0.00,0.00,1.0
mustache,0.00,1.00,0.0


In [8]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.0,1,0,0
glasses,0.0,0,1,0
mustache,0.0,1,0,0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Literal listener


,r1,r2,r3
hat,0.181818,0.818182,0.0
glasses,0.000000,0.000000,1.0
mustache,0.000000,1.000000,0.0



Pragmatic speaker


,hat,glasses,mustache
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Pragmatic listener


,r1,r2,r3
hat,0.330579,0.669421,0.0
glasses,0.000000,0.000000,1.0
mustache,0.000000,1.000000,0.0


In [13]:
    msgs = ['hat', 'glasses', 'mustache', 'tie']
    states = ['r1', 'r2', 'r3', 'r4']
    lex = pd.DataFrame([
        [1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 1.0, 0.0],
        [1.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0, 0.0]], index=msgs, columns=states)
    

In [12]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,r4,costs
hat,1.00,0,0,0,0
glasses,0.00,1,1,0,0
mustache,1.00,0,0,1,0
tie,0.00,1,0,0,0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
hat,1.0,0.0,0.0,0.0
glasses,0.0,0.5,0.5,0.0
mustache,0.5,0.0,0.0,0.5
tie,0.0,1.0,0.0,0.0



Pragmatic speaker


,hat,glasses,mustache,tie
r1,0.666667,0.000000,0.333333,0.000000
r2,0.000000,0.333333,0.000000,0.666667
r3,0.000000,1.000000,0.000000,0.000000
r4,0.000000,0.000000,1.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
hat,1.00,0.00,0.00,0.00
glasses,0.00,0.25,0.75,0.00
mustache,0.25,0.00,0.00,0.75
tie,0.00,1.00,0.00,0.00


In [14]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,r4,costs
hat,1.00,0,0,0,0
glasses,0.00,1,1,0,0
mustache,1.00,0,0,1,0
tie,0.00,1,0,0,0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
hat,1.0,0.0,0.0,0.0
glasses,0.0,0.5,0.5,0.0
mustache,0.5,0.0,0.0,0.5
tie,0.0,1.0,0.0,0.0



Pragmatic speaker


,hat,glasses,mustache,tie
r1,0.666667,0.000000,0.333333,0.000000
r2,0.000000,0.333333,0.000000,0.666667
r3,0.000000,1.000000,0.000000,0.000000
r4,0.000000,0.000000,1.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
hat,1.00,0.00,0.00,0.00
glasses,0.00,0.25,0.75,0.00
mustache,0.25,0.00,0.00,0.75
tie,0.00,1.00,0.00,0.00


In [15]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0], alpha = 1)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,r4,costs
hat,1.00,0,0,0,0
glasses,0.00,1,1,0,0
mustache,1.00,0,0,1,0
tie,0.00,1,0,0,0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
hat,1.0,0.0,0.0,0.0
glasses,0.0,0.5,0.5,0.0
mustache,0.5,0.0,0.0,0.5
tie,0.0,1.0,0.0,0.0



Pragmatic speaker


,hat,glasses,mustache,tie
r1,0.666667,0.000000,0.333333,0.000000
r2,0.000000,0.333333,0.000000,0.666667
r3,0.000000,1.000000,0.000000,0.000000
r4,0.000000,0.000000,1.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
hat,1.00,0.00,0.00,0.00
glasses,0.00,0.25,0.75,0.00
mustache,0.25,0.00,0.00,0.75
tie,0.00,1.00,0.00,0.00


In [16]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0], alpha = 5)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,r4,costs
hat,1.00,0,0,0,0
glasses,0.00,1,1,0,0
mustache,1.00,0,0,1,0
tie,0.00,1,0,0,0
prior,0.25,0.25,0.25,0.25,
alpha,5.00,,,,



Literal listener


,r1,r2,r3,r4
hat,1.0,0.0,0.0,0.0
glasses,0.0,0.5,0.5,0.0
mustache,0.5,0.0,0.0,0.5
tie,0.0,1.0,0.0,0.0



Pragmatic speaker


,hat,glasses,mustache,tie
r1,0.969697,0.000000,0.030303,0.000000
r2,0.000000,0.030303,0.000000,0.969697
r3,0.000000,1.000000,0.000000,0.000000
r4,0.000000,0.000000,1.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
hat,1.000000,0.000000,0.000000,0.000000
glasses,0.000000,0.029412,0.970588,0.000000
mustache,0.029412,0.000000,0.000000,0.970588
tie,0.000000,1.000000,0.000000,0.000000


In [17]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.1, 0.2, 0.3, 0.4], costs=[0.0, 0.0, 0.0, 0.0], alpha = 5)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,r4,costs
hat,1.0,0,0,0,0
glasses,0.0,1,1,0,0
mustache,1.0,0,0,1,0
tie,0.0,1,0,0,0
prior,0.1,0.2,0.3,0.4,
alpha,5.0,,,,



Literal listener


,r1,r2,r3,r4
hat,1.0,0.0,0.0,0.0
glasses,0.0,0.4,0.6,0.0
mustache,0.2,0.0,0.0,0.8
tie,0.0,1.0,0.0,0.0



Pragmatic speaker


,hat,glasses,mustache,tie
r1,0.99968,0.000000,0.00032,0.000000
r2,0.00000,0.010136,0.00000,0.989864
r3,0.00000,1.000000,0.00000,0.000000
r4,0.00000,0.000000,1.00000,0.000000



Pragmatic listener


,r1,r2,r3,r4
hat,1.00000,0.000000,0.000000,0.00000
glasses,0.00000,0.006712,0.993288,0.00000
mustache,0.00008,0.000000,0.000000,0.99992
tie,0.00000,1.000000,0.000000,0.00000


In [18]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.1, 0.7, 9.0, 1.0], alpha = 5)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,r4,costs
hat,1.00,0,0,0,0.1
glasses,0.00,1,1,0,0.7
mustache,1.00,0,0,1,9
tie,0.00,1,0,0,1
prior,0.25,0.25,0.25,0.25,
alpha,5.00,,,,



Literal listener


,r1,r2,r3,r4
hat,1.0,0.0,0.0,0.0
glasses,0.0,0.5,0.5,0.0
mustache,0.5,0.0,0.0,0.5
tie,0.0,1.0,0.0,0.0



Pragmatic speaker


,hat,glasses,mustache,tie
r1,1.510238e-18,0.000000,1.0,0.000000
r2,0.000000e+00,0.006925,0.0,0.993075
r3,0.000000e+00,1.000000,0.0,0.000000
r4,0.000000e+00,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3,r4
hat,1.0,0.000000,0.000000,0.0
glasses,0.0,0.006877,0.993123,0.0
mustache,0.5,0.000000,0.000000,0.5
tie,0.0,1.000000,0.000000,0.0
